In [1]:
from keras.utils import plot_model
from text_analysis.models import CNN_CAE_transfer_module, CNN_module, CNN_CAE_module
import matplotlib.pyplot as plt
import numpy as np
import os
from data_manager import Data_Factory
import tensorflow as tf
import pandas as pd
LOG_DIR = 'logs/doc_embeddings'


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import h5a, h5d, h5ds, h5f, h5fd, h5g, h5r, h5s, h5t, h5p, h5z
/usr/local/lib/python2.7/dist-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p
Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size change

/usr/local/lib/python2.7/dist-packages/sklearn/utils/extmath.py:24: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._logistic_sigmoid import _log_logistic_sigmoid
/usr/local/lib/python2.7/dist-packages/sklearn/utils/extmath.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .sparsefuncs_fast import csr_row_norms


# Load weights

In [2]:
#cnn_cae_transfer
#exp_dir = '/home/wanliz/data/Extended_ctr/convmf/citeulike_a_extended/results/5-8_in-matrix_no-val_0.01-100_w_cnn-100_cae-50_transfer-2_no-sc//fold-4/'
#cnn_cae_concat
#exp_dir ='/home/wanliz/data/Extended_ctr/convmf/citeulike_a_extended/results/6-8_out-of-matrix_no-val_1-1000-w-cnn-100_cae-50_concat/fold-1/'
#cnn
# exp_dir = '/home/wanliz/data/Extended_ctr/convmf/citeulike_a_extended/results/6-8_in-matrix_no-val_0.01-100_w_cnn/fold-1'
exp_dir = '/home/wanliz/data/Extended_ctr/convmf/citeulike_a_extended/results/11-8_in-matrix-200_no-val_0.01-100-w-cnn-100_bn/'
fc_weights_file =os.path.join(exp_dir,'FC_weights.npy')
module = 'cnn'
if module == 'cnn_cae_transfer' or module == 'cnn_cae_concat':
    weights_file = os.path.join(exp_dir, 'CNN_CAE_weights.hdf5')
elif module == 'cnn':
    weights_file = os.path.join(exp_dir, 'CNN_weights.hdf5')

    
if module == 'cnn_cae_transfer':
    model = CNN_CAE_transfer_module(output_dimesion= 200, vocab_size=8001, dropout_rate=0.2,
                                emb_dim=200, max_len=300, nb_filters=100, init_W=None, cae_N_hidden=50,
                                   nb_features=17)
elif module =='cnn_cae_concat':
    model =CNN_CAE_module(output_dimesion= 200, vocab_size=8001, dropout_rate=0.2,
                                emb_dim=200, max_len=300, nb_filters=100, init_W=None, cae_N_hidden=50,
                                   nb_features=17)
elif module == 'cnn':
    model = CNN_module(output_dimesion=200, vocab_size=8001, dropout_rate=0.2,
                                    emb_dim=200, max_len=300, nb_filters=100, init_W=None)
    
if not os.path.exists(fc_weights_file):
    model.load_model(weights_file)
    if module == 'cnn_cae_concat':
        layer_name = 'joint_output'
    else:
        layer_name = 'fully_connect'
    weights = model.model.get_layer(layer_name).get_weights()
    bias = weights[1]
    fc_weights = weights[0]
    np.save(fc_weights_file,fc_weights)
else:
    fc_weights = np.load(fc_weights_file)


# Load data

In [2]:
aux_path = '/home/wanliz/data/Extended_ctr/convmf/citeulike_a_extended/preprocessed'
data_factory = Data_Factory()
D_all = data_factory.load_documents(aux_path)
CNN_X = D_all['X_sequence']
len(CNN_X)
theta = model.get_projection_layer(CNN_X)
theta.shape

Load preprocessed document data - /home/wanliz/data/Extended_ctr/convmf/citeulike_a_extended/preprocessed/document.all


NameError: name 'model' is not defined

# Save doc_embeddings meta-data

In [4]:
# Read raw data, turns out there is 20 missing documents info 
#paper_info = '/home/wanliz/data/Extended_ctr/convmf/citeulike_a_extended/raw/papers_info_corrected_pages_years.csv'
#df = pd.read_csv(paper_info,sep=',')
# Select the ones you want
# df1 = df[['citeulike_id','title']]
# df1.to_csv(os.path.join(LOG_DIR, 'metadata_2.tsv'), sep='\t',index=False)

paper_info = '/home/wanliz/data/Extended_ctr/convmf/citeulike_a_extended/raw/papers_info_corrected_pages_years.csv.cleaned'
labels = ['citeulike_id','title']
labels_dtype = { 'title': str}
df = pd.read_table(paper_info, delimiter='\t', usecols=labels,dtype=labels_dtype,na_values='\\N',na_filter=False)
df.to_csv(os.path.join(LOG_DIR, 'metadata.tsv'), sep='\t',index=False)


/usr/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Save word_embeddings meta-data (word, word_frequency)

In [15]:
with open(os.path.join(LOG_DIR, 'vocab.tsv'), 'w') as filehandle:  
    filehandle.write('%s\t%s\n' % ('Word', 'frequency'))
    filehandle.write('%s\t%d\n' % ('unk', -1))
    for listitem in D_all['X_vocab']:
        filehandle.write('%s\t%d\n' % (listitem[0].encode('utf-8'), listitem[1]))

# Load wrod embeddings

In [3]:
pretrain_w2v = '/home/wanliz/data/cbow_w2v/w2v_200.txt'
init_W = data_factory.read_pretrained_word2vec(pretrain_w2v, D_all['X_vocab'], 200)


7963 words exist in the given pretrained model


In [9]:
print (len(D_all['X_vocab']))
init_W.shape

8000


(8001, 200)

In [12]:
print (D_all['X_vocab'])
init_W

[(u'00', 0), (u'000', 1), (u'001', 2), (u'01', 3), (u'02', 4), (u'034', 5), (u'039', 6), (u'05', 7), (u'08', 8), (u'10', 9), (u'100', 10), (u'1000', 11), (u'1002', 12), (u'101', 13), (u'105', 14), (u'106', 15), (u'107', 16), (u'1073', 17), (u'108', 18), (u'109', 19), (u'1093', 20), (u'11', 21), (u'110', 22), (u'1101', 23), (u'1126', 24), (u'1177', 25), (u'12', 26), (u'120', 27), (u'124', 28), (u'13', 29), (u'14', 30), (u'140', 31), (u'146', 32), (u'147', 33), (u'148', 34), (u'15', 35), (u'150', 36), (u'151', 37), (u'16', 38), (u'160', 39), (u'16s', 40), (u'17', 41), (u'18', 42), (u'180', 43), (u'183', 44), (u'19', 45), (u'1960s', 46), (u'1970s', 47), (u'1980', 48), (u'1980s', 49), (u'1983', 50), (u'1985', 51), (u'1986', 52), (u'1987', 53), (u'1988', 54), (u'1989', 55), (u'1990', 56), (u'1990s', 57), (u'1991', 58), (u'1992', 59), (u'1993', 60), (u'1994', 61), (u'1995', 62), (u'1996', 63), (u'1997', 64), (u'1998', 65), (u'1999', 66), (u'20', 67), (u'200', 68), (u'2000', 69), (u'2001', 70

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.46545 ,  1.358249,  0.452584, ...,  0.404637,  0.395495,
         1.062556],
       [-1.350717,  0.314082,  1.157526, ...,  3.028673,  0.535382,
         0.26888 ],
       ...,
       [-1.501189,  3.976764,  1.742986, ..., -1.928872, -1.114995,
         2.26562 ],
       [ 0.449416,  2.796112,  1.370877, ..., -3.412091, -0.823349,
         1.894279],
       [-1.325043,  0.62969 ,  0.975433, ..., -1.077265, -0.522523,
         0.72491 ]])

# Visualize embeddeings

In [7]:
from tensorflow.contrib.tensorboard.plugins import projector

tf.reset_default_graph()
doc_embeddings = tf.get_variable('doc_embeddings',shape=theta.shape,
                                 initializer=tf.constant_initializer(theta),
                                 trainable=False)
word_embeddings =  tf.get_variable('word_embeddings',shape=init_W.shape,
                                 initializer=tf.constant_initializer(init_W),
                                 trainable=False)
sess = tf.Session()  # Create new session
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()

# Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
'''docs embeddings'''
embedding = config.embeddings.add()
embedding.tensor_name = doc_embeddings.name
# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path =  'metadata.tsv'
'''words embeddings'''
embedding = config.embeddings.add()
embedding.tensor_name = word_embeddings.name
# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path =  'vocab.tsv'

# Use the same LOG_DIR where you stored your checkpoint.
summary_writer = tf.summary.FileWriter(LOG_DIR)

# The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
# read this file during startup.
projector.visualize_embeddings(summary_writer, config)
saver.save(sess,os.path.join(LOG_DIR, 'model.ckpt' ))

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/cluster/supervised.py:23: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .expected_mutual_info_fast import expected_mutual_information
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/pairwise.py:30: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .pairwise_fast import _chi2_kernel_fast, _sparse_manhattan


'logs/doc_embeddings/model.ckpt'